In [1]:
from pyscf import gto, scf, ao2mo, fci
import CC, Wick


In [2]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff)

cisolver = fci.FCI(mol, mf.mo_coeff)

In [3]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [4]:
H2CCSD = CC.coupledCluster(mf, range(1,3))

2


In [9]:
H2CCSDExcitations = H2CCSD[0]
H2CCSDAmplitudes = H2CCSD[1]
H2CCSDClusterOperator = CC.getClusterOperator(H2CCSDExcitations, H2CCSDAmplitudes)

TypeError: 'operatorSum' object is not subscriptable

In [24]:
print(H2CCSDClusterOperator)

-0.0
 + -4.394149808727142e-17 * a^{1\alpha}a_{0\alpha}
 + -4.394149808727142e-17 * a^{1\beta}a_{0\beta}
 + -0.06883332874447325 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.06883332874447325 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}


In [25]:
hamiltonian = CC.get1bodyHamiltonian(mf) + CC.get2bodyHamiltonian(mf)

In [26]:
waveOperator = CC.exponentialOperator(H2CCSDClusterOperator, 4)
inverseWaveOperator = CC.exponentialOperator((-1) * H2CCSDClusterOperator, 4)

In [27]:
CCSDEnergy = Wick.vacuumExpectationValue(inverseWaveOperator * hamiltonian * waveOperator, [1,0]) + mol.energy_nuc()
print(CCSDEnergy)

-1.1284543355083052


In [28]:
print(CC.BCHSimilarityTransform(hamiltonian, CC.spinFreeNtupleExcitation((1,1), (0,0)), 4))

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.0
 + 0.0
 + 0.0
 + 0.0
 + 0.0
 + 0.3226247132942424 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 0.0
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0
 + 0.0
 + 0.0
 + 0.0
 + 4.163336342344337e-17 * a^{0\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.3226247132942424 * a^{0\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 2.7755575615628914e-17 * a^{0\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.0
 + 0.3226247132942424 * a^{0\beta}a^{1\beta}a_

In [29]:
print(hamiltonian)

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + 0.0
 + 0.0
 + 0.0
 + 0.0
 + 0.3270047556780619 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 5.204170427930421e-17 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{0\alpha}
 + 0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\alpha}a_{0\alpha}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942424 * a^{0

In [30]:
print(CC.spinFreeNtupleExcitation((1,1), (0,0)).conjugate() * CC.BCHSimilarityTransform(hamiltonian, CC.spinFreeNtupleExcitation((1,1), (0,0)), 4))

0.0
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}
 + -4.7142356915405136e-17 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a_{1\beta}
 + 0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{0\alpha}a^{0\beta}a_{1\b

In [31]:
print(CC.spinFreeDoubleExcitation(1,2,3,4))

1.0 * a^{1\alpha}a^{2\alpha}a_{4\alpha}a_{3\alpha}
 + 1.0 * a^{1\alpha}a^{2\beta}a_{4\beta}a_{3\alpha}
 + 1.0 * a^{1\beta}a^{2\alpha}a_{4\alpha}a_{3\beta}
 + 1.0 * a^{1\beta}a^{2\beta}a_{4\beta}a_{3\beta}


Larger basis set calculation

In [32]:
mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = '6-31g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

cisolver = fci.FCI(mol, mf.mo_coeff)

In [33]:
cisolver.kernel()

(-1.1461680985775962,
 array([[ 9.90466208e-01,  1.31838984e-15,  8.70605454e-03,
          3.41814448e-16],
        [ 1.40666155e-15, -9.91540669e-02, -1.18751575e-16,
         -5.11213337e-02],
        [ 8.70605454e-03, -1.46893055e-16, -4.48764561e-02,
         -8.66372876e-17],
        [ 3.04169713e-16, -5.11213337e-02, -9.25545999e-17,
         -4.18675608e-02]]))

In [34]:
H2CCSD = CC.coupledCluster(mf, range(1,3))

9


In [ ]:
H2CCSDExcitations = H2CCSD[0]
H2CCSDAmplitudes = H2CCSD[1]
H2CCSDClusterOperator = CC.getClusterOperator(H2CCSDExcitations, H2CCSDAmplitudes)

In [ ]:
print(H2CCSDClusterOperator)

In [ ]:
hamiltonian = CC.get1bodyHamiltonian(mf) + CC.get2bodyHamiltonian(mf)

In [ ]:
waveOperator = CC.exponentialOperator(H2CCSDClusterOperator, 4)
inverseWaveOperator = CC.exponentialOperator((-1) * H2CCSDClusterOperator, 4)

In [ ]:
CCSDEnergy = Wick.vacuumExpectationValue(inverseWaveOperator * hamiltonian * waveOperator, vacuum) + mol.energy_nuc()
print(CCSDEnergy)

In [ ]:
mol = gto.Mole()
mol.verbose = 1
mol.atom = 'Be 0 0 0'
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

cisolver = fci.FCI(mol, mf.mo_coeff)

In [ ]:
cisolver.kernel()

(-1.1461680985775962,
 array([[ 9.90466208e-01,  1.31838984e-15,  8.70605454e-03,
          3.41814448e-16],
        [ 1.40666155e-15, -9.91540669e-02, -1.18751575e-16,
         -5.11213337e-02],
        [ 8.70605454e-03, -1.46893055e-16, -4.48764561e-02,
         -8.66372876e-17],
        [ 3.04169713e-16, -5.11213337e-02, -9.25545999e-17,
         -4.18675608e-02]]))

In [ ]:
BeCCSD = CC.coupledCluster(mf, range(1,3))

9


In [ ]:
BeCCSDExcitations = BeCCSD[0]
BeCCSDAmplitudes = BeCCSD[1]
BeCCSDClusterOperator = CC.getClusterOperator(BeCCSDExcitations, BeCCSDAmplitudes)

In [ ]:
print(BeCCSDClusterOperator)

In [ ]:
hamiltonian = CC.get1bodyHamiltonian(mf) + CC.get2bodyHamiltonian(mf)

In [ ]:
waveOperator = CC.exponentialOperator(BeCCSDClusterOperator, 4)
inverseWaveOperator = CC.exponentialOperator((-1) * BeCCSDClusterOperator, 4)

In [ ]:
CCSDEnergy = Wick.vacuumExpectationValue(inverseWaveOperator * hamiltonian * waveOperator, vacuum) + mol.energy_nuc()
print(CCSDEnergy)